In [12]:
from langchain_core.documents import Document

In [13]:
doc = Document (
    page_content = 'this is the main text content Iam using to Create RAG',
    metadata ={
        "source" : "example.txt",
        "pages" : 1,
        "author": "Hema",
        "date_created" : "02-18-2026"
    }
)
doc

Document(metadata={'source': 'example.txt', 'pages': 1, 'author': 'Hema', 'date_created': '02-18-2026'}, page_content='this is the main text content Iam using to Create RAG')

In [14]:
## create a simple txt file
import os
os.makedirs("../data/text_files", exist_ok = True)

In [15]:
sample_texts = {
    "../data/text_files/rag_intro.txt": 
    """
    Retrieval-Augmented Generation (RAG) is an architectural framework designed to enhance the accuracy and reliability of Large Language Models (LLMs) by grounding them in external, authoritative knowledge bases. While standard models rely strictly on the data they were originally trained on—which can lead to outdated information or "hallucinations"—RAG enables a model to look up fresh data before generating a response. This process involves retrieving relevant document snippets from a source like a private database or the live web, and then passing that context into the model alongside the user’s original question.""" ,
   
   "../data/text_files/langchain_intro.txt":  """ Langchain Intro

LangChain serves as the primary "operating system" or toolkit for building these RAG systems. It provides a modular library of "LEGO-like" components—such as document loaders, text splitters, and vector store connectors—that simplify the complex plumbing required to link an LLM to external data. Instead of developers manually coding every step of the retrieval process, LangChain allows them to create "chains" that automate the flow of information from ingestion to the final generated answer. 
The synergy between the two lies in how LangChain orchestrates the RAG pipeline. Using LangChain, a developer can quickly load diverse data formats like PDFs or website content, break them into manageable chunks, and store them as mathematical "embeddings" in a vector database for rapid searching. When a user asks a question, LangChain's retrieval modules find the most relevant pieces of information and prompt the LLM to use that specific data to craft a factually grounded and contextually correct response. 
"""
}

for filepath,content in sample_texts.items():
    with open(filepath , 'w' , encoding = "utf-8") as f:
        f.write(content)
print("sample text file created")

sample text file created


In [16]:
##Text Loader

from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import TextLoader

loader = TextLoader ("../data/text_files/rag_intro.txt" , encoding = "utf-8")
document = loader.load()
print(document)

[Document(metadata={'source': '../data/text_files/rag_intro.txt'}, page_content='\n    Retrieval-Augmented Generation (RAG) is an architectural framework designed to enhance the accuracy and reliability of Large Language Models (LLMs) by grounding them in external, authoritative knowledge bases. While standard models rely strictly on the data they were originally trained on—which can lead to outdated information or "hallucinations"—RAG enables a model to look up fresh data before generating a response. This process involves retrieving relevant document snippets from a source like a private database or the live web, and then passing that context into the model alongside the user’s original question.')]


In [17]:
###Directory Loader ## Another way to load the data 
from langchain_community.document_loaders import DirectoryLoader

#Load all the text files from teh directory

dir_loader = DirectoryLoader(
    "../data/text_files",
    glob = "**/*.txt" ,## Pattern to match files
    loader_cls = TextLoader , #Loader class to use # if it is pdf loader use pdf 
    loader_kwargs = {'encoding' : 'utf-8'},
    show_progress = False
)

documents = dir_loader.load()
documents

[Document(metadata={'source': '..\\data\\text_files\\langchain_intro.txt'}, page_content=' Langchain Intro\n\nLangChain serves as the primary "operating system" or toolkit for building these RAG systems. It provides a modular library of "LEGO-like" components—such as document loaders, text splitters, and vector store connectors—that simplify the complex plumbing required to link an LLM to external data. Instead of developers manually coding every step of the retrieval process, LangChain allows them to create "chains" that automate the flow of information from ingestion to the final generated answer. \nThe synergy between the two lies in how LangChain orchestrates the RAG pipeline. Using LangChain, a developer can quickly load diverse data formats like PDFs or website content, break them into manageable chunks, and store them as mathematical "embeddings" in a vector database for rapid searching. When a user asks a question, LangChain\'s retrieval modules find the most relevant pieces of

In [ ]:

from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader

#Load all the pdf files from the directory

from langchain_community.document_loaders import DirectoryLoader, PyMuPDFLoader

dir_loader = DirectoryLoader(
    "../data/pdf_files",
    glob="**/*.pdf",
    loader_cls=PyMuPDFLoader,
    show_progress=True
)

pdf_documents = dir_loader.load()
print(pdf_documents)

100%|██████████| 3/3 [00:00<00:00, 78.80it/s]

[Document(metadata={'producer': 'Skia/PDF m145 Google Docs Renderer', 'creator': '', 'creationdate': '', 'source': '..\\data\\pdf_files\\Hema_Resume_Business_Analyst.pdf', 'file_path': '..\\data\\pdf_files\\Hema_Resume_Business_Analyst.pdf', 'total_pages': 1, 'format': 'PDF 1.4', 'title': 'Hema_Resume_Business_Analyst', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': '', 'page': 0}, page_content='Hemasree Uppaluri Venkata Syamala \n                                       uvs.hema@gmail.com | 602-813-8372 | LinkedIn| GitHub \n \nEDUCATION \nMaster’s of Science in Information Technology                                                                                                         Aug’22 – May’24              \nArizona State University | Arizona US \nCoursework: Cloud Computing, Data Visualization, Statistical Machine Learning, Big Data Analytics, Project Management, Deep Learning, Natural \nLanguage Processing \n \nTECHNIC

  Second step in the process 
  Chuncking the data

firstly, we load the pdfs we have uploaded on our system


In [19]:
from pathlib import Path
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 3 PDF files to process

Processing: Hema_Resume_Business_Analyst.pdf
  ✓ Loaded 1 pages

Processing: Hema_Resume_Data Analyst.pdf
  ✓ Loaded 1 pages

Processing: Hema_Resume_Data Engineer.pdf
  ✓ Loaded 1 pages

Total documents loaded: 3


In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_documents(documents, chunk_size=500, chunk_overlap=50):
    """ Split documents into smaller chunks
    Args:
        documents: List of Document objects or raw strings.
        chunk_size: Max characters per chunk.
        chunk_overlap: Overlap between chunks.
    Returns:
        List of Document chunks
    """
    # Ensure all inputs are Document objects
    if isinstance(documents[0], str):
        documents = [Document(page_content=doc, metadata={}) for doc in documents]

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=['\n\n', '\n', ' ', '']
    )
    
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # show example of chunk
    if split_docs:
        print(f"\n Example chunk")
        print(f"Content : {split_docs[0].page_content[:200]}...")
        print(f"Metadata : {split_docs[0].metadata}")
    
    return split_docs


chunks = split_documents(all_pdf_documents)
print(f"Total chunks created: {len(chunks)}")

Split 3 documents into 46 chunks

 Example chunk
Content : Hemasree  Uppaluri  Venkata  Syamala                                         uvs.hema@gmail.com |  602-813-8372  |  LinkedIn|  GitHub...
Metadata : {'producer': 'Skia/PDF m145 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Hema_Resume_Business_Analyst', 'source': '..\\data\\pdf_files\\Hema_Resume_Business_Analyst.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'Hema_Resume_Business_Analyst.pdf', 'file_type': 'pdf'}
Total chunks created: 46


In [21]:
##Embedding and vectorStoreDB


In [27]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [34]:
from typing import List


class EmbeddingManger:
    """
    Handles document embedding generation using SentenceTransformer
    """
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"): # text into vectors
        """
        Initialize the embedding manager

        Args:

            model_name = HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()
    
    def _load_model(self):
        """
        Load the SentenceTransformer model
        """
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"model loaded successfully .Embedding Dimension : {self .model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name} : {e}")
            raise
    def generate_embeddings(self , texts : List[str]) -> np.ndarray:
        
        """
        Generate embeddings for a list of texts 

        Args: 
            text : list of text strings to embed
        
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)

        """
        if not self.model:
            raise ValueError("Model not loaded")
        print (f"Generating embeddings for {len (texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar = True)
        print(f"Generated embeddings with shape : {embeddings.shape}")
        return embeddings
    



Intialize the embedding manager

In [35]:
embedding_manager = EmbeddingManger()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
model loaded successfully .Embedding Dimension : 384


Vector store

In [ ]:
class Vectorstore:
    """Manages document embeddings in a chromadb vector store"""
    def __init__(self , collection_name :str = 'pdf_documents' , persist_directory : str = "../data/vector_store"):

        """ Intialize the vector store
        Args:
            collection_name : Name of the chromadb collection
            persist_directory : Directory to persist the vectore store
            """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Intiliaze chromadb client and collection"""
        try:
            #create persistent chromadb client
            os.makedirs(self.persist_directory , exist_ok=True)
            self.client = chromadb.PersistentClient(path = self.persist_directory)

            # get or create collection
            #where we are going to store all teh vectors
            self.collection = self.client.get_or_create_collection(
                name = self.collection_name,
                metadata= {"description" : "PDF Document embeddings for RAG"}
            )
            print(f"vector store intialized . collection : {self.collection_name}")
            print(f" Existing documents in collection : {self.collection.count()}")
        except Exception as e:
            print(f"Error Initializing vectore store : {e}")
            raise

    def add_documents(self, documents : List[Any], embeddings : np.ndarray):
        """Add documents and their embeddings to the vectore store
            Args:
            documents : List of Langchain documents
            embeddings : Corresponding embeddings for the documents
            """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        print(f"Adding {len(documents)} documents to vector store ")

        #prepare data for chromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []

        for i ,(doc , embedding) in enumerate(zip(documents , embeddings)):
            #Generate unique ID 
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}" # id for specific variable
            ids.append(doc_id)

            #prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length']= len(doc.page_content)
            metadatas.append(metadata)

            #document content
            documents_text.append(doc.page_content)

            #embedding
            embeddings_list.append(embedding.tolist())

            # Add to collection

            try:
                self.collection.add(
                    ids = ids,
                    embeddings= embeddings_list,
                    metadatas=metadatas,
                    documents= documents_text
                )
                print(f"successfully added {len(documents)} documents to vector store")
                print(f"Total documents in collection : {self.collection.count()}")
            except Exception as e:
                print(f"Error adding documents to vector store ; {e}")

Vectorstore = Vectorstore()
Vectorstore

#to add documents call add_document def



vector store intialized . collection : pdf_documents
 Existing documents in collection : 0


In [39]:
### convert the text to embeddings
texts = [doc.page_content for doc in chunks]
texts

##generate the embeddings

embeddings = embedding_manager.generate_embeddings(texts)

##store in the vector database
Vectorstore.add_documents(chunks, embeddings)

Generating embeddings for 46 texts...


Batches: 100%|██████████| 2/2 [00:00<00:00,  2.69it/s]


Generated embeddings with shape : (46, 384)
Adding 46 documents to vector store 
successfully added 46 documents to vector store
Total documents in collection : 1
successfully added 46 documents to vector store
Total documents in collection : 2
successfully added 46 documents to vector store
Total documents in collection : 3
successfully added 46 documents to vector store
Total documents in collection : 4
successfully added 46 documents to vector store
Total documents in collection : 5
successfully added 46 documents to vector store
Total documents in collection : 6
successfully added 46 documents to vector store
Total documents in collection : 7
successfully added 46 documents to vector store
Total documents in collection : 8
successfully added 46 documents to vector store
Total documents in collection : 9
successfully added 46 documents to vector store
Total documents in collection : 10
successfully added 46 documents to vector store
Total documents in collection : 11
successfully ad

Retriver Pipeline From Vectorstore

In [47]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""

    def __init__(self ,vector_store: Vectorstore ,embedding_managermbedding_manager :EmbeddingManger):
        """Intialize the retriever
        Args
            vectore_store = vector store containing document embeddings
            embeddings_manager :Manager for generating query embeddings
            """
        self.vector_store = Vectorstore
        self.embedding_manager = embedding_manager

        ##retrive the data from specific query

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query

        Args:
            query : The search query
            top_k :Number of top results to return
            score_threshold :Minimum similarity score threshold
        Returns: 
            List of dictionaries containg retrived documents and metadata
        """
        print(f"Retrieving documents for query : '{query}'")
        print (f"Top K : {top_k} , Score threshold : {score_threshold}")

        #generate query embedding
        query_embeddings = self.embedding_manager.generate_embeddings([query])[0]

        # search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings = [query_embeddings.tolist()],
                n_results = top_k
            )

            #process results
            retrived_docs= []

            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results ['distances'][0]
                ids = results ['ids'][0]

                for i , (doc_id , document , metadata , distance) in enumerate(zip(ids, documents , metadatas , distances)):
                    #convert the distance to similarity score(Chromadb uses cosine distance)
                    similarity_score = 1- distance

                    if similarity_score >= score_threshold:
                        retrived_docs .append(
                            {
                                'id' : doc_id,
                                'content' :document,
                                'metadata' : metadata,
                                'similarity_score' : similarity_score,
                                'distance' : distance,
                                'rank' :i+1})
                print(f"Retrieved {len(retrived_docs)} documents (after filetring)")
                   
            else:
                print("No documents found")
            return retrived_docs
        except Exception as e:
            print(f"Error during retrievak : {e}")
            return[]
rag_retriver = RAGRetriever (Vectorstore , EmbeddingManger)
rag_retriver

In [48]:
rag_retriver.retrieve("can you give me technical skills")

Retrieving documents for query : 'can you give me technical skills'
Top K : 5 , Score threshold : 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 30.17it/s]

Generated embeddings with shape : (1, 384)
Retrieved 3 documents (after filetring)


[{'id': 'doc_5ff19717_2',
  'content': 'TECHNICAL  SKILLS  Programming :  Python,  SQL,  HTML,  CSS  Frameworks/Tools:  Pandas,  NumPy,  Scikit-learn,  NLTK,  Flask,  Tableau,  Power  BI,  SSIS,  Git,  VS  Code  Cloud  Technologies  and  Databases:  AWS  (S3,  EC2),  Azure,  Postgresql,  Oracle,  Microsoft  SQL  Server,  Google  Big  Query,  Google  cloud  functions  AI  Practices:  Natural  Language  Processing  (NLP),  Sentiment  Analysis,  Text  Classification,  Regression  Analysis,  Data  Visualization,  Predictive',
  'metadata': {'creationdate': '',
   'content_length': 485,
   'page_label': '1',
   'source': '..\\data\\pdf_files\\Hema_Resume_Business_Analyst.pdf',
   'producer': 'Skia/PDF m145 Google Docs Renderer',
   'doc_index': 2,
   'source_file': 'Hema_Resume_Business_Analyst.pdf',
   'file_type': 'pdf',
   'total_pages': 1,
   'title': 'Hema_Resume_Business_Analyst',
   'creator': 'PyPDF',
   'page': 0},
  'similarity_score': 0.055256783962249756,
  'distance': 0.9447432